In [2]:
!pip install langchain_community
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [3]:
!pip install langchain-huggingface

In [4]:
!pip install langchain

In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 8.9 MB/s eta 0:00:00


In [6]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 21.9 MB/s eta 0:00:00


In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [8]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.5 MB/s eta 0:00:00


In [11]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("/content/sample_data/us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': '/content/sample_data/us_census/p70-178.pdf', 'page': 0, 'page_label': '1'}, page_content='Occupation, Earnings, and Job \nCharacteristics\nJuly 2022\nP70-178\nClayton Gumber and Briana Sullivan\nCurrent Population Reports\nINTRODUCTION\nWork is a critical component of our lives and provides \na way to obtain material and nonmonetary benefits \nlike employer-provided health insurance. Scholars \nsuggest that our identities are also tied to the notion \nof “what we do” (Christiansen, 1999), and that who \nwe are is determined partly by our occupational iden -')

In [12]:
len(final_documents)

793

In [13]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    #model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

<ipython-input-13-4d13835eae7c>:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings=HuggingFaceBgeEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[ 9.61705018e-03  3.12648676e-02 -3.82289663e-02 -2.86026276e-03
  3.51881683e-02  5.32390736e-02  1.00248463e-01 -2.50928979e-02
 -1.51269250e-02 -7.88904261e-03  4.51828539e-02  1.44510306e-02
 -2.69028172e-02 -3.12521271e-02 -3.68831418e-02  4.77155577e-03
  1.00345677e-03  4.28187363e-02 -5.39846718e-02  2.44278517e-02
 -3.75660416e-03  9.41762328e-03 -4.68625501e-02 -2.96772663e-02
  6.15552776e-02  3.06337606e-02  2.48638855e-04  2.06298511e-02
  1.68206915e-02 -7.12122247e-02 -1.27254603e-02  3.32844853e-02
  4.24330495e-02  1.59378126e-02  6.65532574e-02  3.07344384e-02
 -1.40622174e-02  7.35096559e-02 -3.47210997e-04 -4.70641907e-03
  1.69250071e-02 -2.29726508e-02 -5.29297553e-02  1.09618844e-03
 -2.29420941e-04  8.98156539e-02 -1.42218750e-02 -2.69810949e-02
 -8.91320780e-02 -6.26153033e-03  2.99166348e-02  6.75390381e-03
  1.70562025e-02  1.25144869e-01  2.83732405e-03  8.23831826e-04
  4.22866419e-02  1.25784264e-03 -2.59374995e-02 -1.71486624e-02
  9.08519886e-03  2.88294

In [15]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [16]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

and technical occupations.
 • Over 91 percent of all workers 
were paid a wage or salary. 
Workers can also receive com-
mission, tips, overtime, and/
or bonus pay, either to supple-
ment their wage or salary or as 
stand-alone earnings.
 • A majority of workers with 
private health insurance cover-
age (approximately 86 percent) 
were covered by an employer-
provided health insurance plan. 
Among workers receiving this 
type of coverage, about 73 
percent in the private sector,


In [17]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7a4fe463a750> search_kwargs={'k': 3}


In [30]:
from google.colab import userdata
sec_key=userdata.get("HUGGINGFACEHUB_API_TOKEN")
print(sec_key)

SecretNotFoundError: Secret HUGGINGFACEHUB_API_TOKEN does not exist.

In [19]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=sec_key

In [31]:
## Environment secret keys
from google.colab import userdata
sec_key=userdata.get("HF_TOKEN")
print(sec_key)


SecretNotFoundError: Secret HF_TOKEN does not exist.

In [21]:
import os
os.environ["HF_TOKEN"] = sec_key


In [22]:
from langchain_huggingface import HuggingFaceEndpoint

In [23]:
repo_id="mistralai/Mistral-7B-Instruct-v0.2"
llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=128,temperature=0.7,token=sec_key)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [24]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [25]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [27]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [28]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [29]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

1. The context does not provide enough information to determine the differences in the uninsured rate by state in 2022. The context focuses on the demographics and employment characteristics of workers, including their insurance coverage, but it does not provide data on the uninsured rate by state for that year.

 2. According to the context, in 2018, approximately 86 percent of workers were covered by an employer-provided health insurance plan. Among these workers, 73 percent in the private sector, 82 percent in government jobs, and 37 percent of the self-employed were policyholders.

 3. The context also mentions that data for this report comes from the 2018 Survey of Income and Program Participation (SIPP) and the American Community Survey (ACS), but it does not specify which survey was used to gather information on the uninsured rate by state in 2022.

 4. To answer the question, additional information would be needed, such as data from a reliable source on the uninsured rate by st